In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam
from keras.metrics import F1Score, Recall, Precision
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
df = pd.read_csv('train.csv')

df['ID'] = df['ID'].apply(lambda x: int(x, 16))

df['Name'] = df['Name'].str.strip().str.lower()
df['Occupation'] = df['Occupation'].str.strip().str.lower()

df = pd.get_dummies(df, columns=['Customer_ID', 'Month', 'Occupation', 'Type_of_Loan', 'Payment_of_Min_Amount'], drop_first=True)

for column in df.select_dtypes(include=[np.number]).columns:
    df[column].fillna(df[column].mean(), inplace=True)

for column in df.select_dtypes(include=['object']).columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

X = df.drop(columns=['Credit_Score'])  
y = df['Credit_Score'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Проверка на наличие NaN в X_train:", X_train.isnull().sum().sum())
print("Проверка на наличие NaN в y_train:", y_train.isnull().sum())
print("Проверка на наличие NaN в X_test:", X_test.isnull().sum().sum())
print("Проверка на наличие NaN в y_test:", y_test.isnull().sum())

df.head()

C:\Users\Камилла\AppData\Local\Temp\ipykernel_15504\773907406.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


Проверка на наличие NaN в X_train: 0
Проверка на наличие NaN в y_train: 0
Проверка на наличие NaN в X_test: 0
Проверка на наличие NaN в y_test: 0


,ID,Name,Age,SSN,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,...,"Type_of_Loan_Student Loan, and Credit-Builder Loan","Type_of_Loan_Student Loan, and Debt Consolidation Loan","Type_of_Loan_Student Loan, and Home Equity Loan","Type_of_Loan_Student Loan, and Mortgage Loan","Type_of_Loan_Student Loan, and Not Specified","Type_of_Loan_Student Loan, and Payday Loan","Type_of_Loan_Student Loan, and Personal Loan","Type_of_Loan_Student Loan, and Student Loan",Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes
0,-1.731993,aaron maashoh,23,821-00-0265,19114.12,-8.072221e-01,-0.120024,-0.14315,-0.148934,4,...,False,False,False,False,False,False,False,False,True,False
1,-1.731970,aaron maashoh,23,821-00-0265,19114.12,-3.098619e-16,-0.120024,-0.14315,-0.148934,4,...,False,False,False,False,False,False,False,False,True,False
2,-1.731947,aaron maashoh,-500,821-00-0265,19114.12,-3.098619e-16,-0.120024,-0.14315,-0.148934,4,...,False,False,False,False,False,False,False,False,True,False
3,-1.731924,aaron maashoh,23,821-00-0265,19114.12,-3.098619e-16,-0.120024,-0.14315,-0.148934,4,...,False,False,False,False,False,False,False,False,True,False
4,-1.731901,aaron maashoh,23,821-00-0265,19114.12,-8.072221e-01,-0.120024,-0.14315,-0.148934,4,...,False,False,False,False,False,False,False,False,True,False


In [12]:
input_dim = df.shape[1] - 1
num_classes = df['Credit_Score'].nunique()

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [13]:
num_features = df.shape[1] - 1
print("Количество признаков:", num_features)

num_classes = df['Credit_Score'].nunique()
print("Количество классов:", num_classes)

Количество признаков: 18804
Количество классов: 3


In [14]:
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=[F1Score(), Recall(), Precision()])

In [15]:
model.fit(X_train, y_train, epochs=10, batch_size=32)
model.save('model.h5')

MemoryError: Unable to allocate 11.2 GiB for an array with shape (18804, 80000) and data type object